In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import re
from sklearn import preprocessing
import datetime

In [ ]:
class Plic_import_error(Exception):
    pass


In [ ]:
class Plic_importer:
    nan_value = -1
    sep = ","
    index_col = 0
    
    def __init__(self, file_name):
        self.file_name = file_name
        self.df = None
        self.map_dict = {}
        
    def excel_to_pandas_df(self):
        return pd.read_excel(self.file_name, index_col=self.index_col).fillna(self.nan_value)       
        
    def csv_to_pandas_df(self):
        return pd.read_csv(self.file_name, index_col=self.index_col, sep=self.sep).fillna(self.nan_value) 
    
    def drop_empty_cols(self):
        cols_to_drop = []
        
        for i in self.df.columns.values:
            o = self.df[i].unique()
            if len(o) == 1 and o[0] == -1:
                cols_to_drop.append(i)
        
        self.df.drop(cols_to_drop, axis=1, inplace=True)
                
    def visit_cols_to_rows(self):
        suffixes = ["_%s" % a for a in range(0,10)] + \
           ["_%s_a" % a for a in range(0,10)] + \
           ["_%s_recod" % a for a in range(0,10)]
        
        mv_cols = [x for x in self.df.columns if x.endswith(tuple(suffixes))]
        perdurant_cols = [x for x in self.df.columns if x not in mv_cols]
        
        new_cols = set()
        
        for col in mv_cols:
            for s in suffixes:
                col = col.replace(s, "")
            col.replace("__", "_")
            if col.endswith("_"):
                col = col[:-1]
            new_cols.add(col)

        for col in perdurant_cols:
            new_cols.add(col)

        new_cols.add("cod_pz")
        
        fields_per_visit = set()

        for col in mv_cols:
            for s in suffixes:
                col = col.replace(s, re.sub("\d", "%s", s))
            fields_per_visit.add(col)
            
        single = dict()

        for col in mv_cols:
            _col = col
            for s in suffixes:
                col = col.replace(s, "")
            col.replace("__", "_")
            if col.endswith("_"):
                col = col[:-1]
            single[_col] = col
        
        new_data = []

        for paz in self.df.index.values:
                obj = self.df.loc[paz]
                for visit in range(1, 5):
                    this = {"cod_pz": paz}
                    for old_col in fields_per_visit:
                        try:
                            this[single[old_col % visit]] = obj[old_col % visit]
                        except KeyError:
                            pass
                    for pd_col in perdurant_cols:
                        this[pd_col] = obj[pd_col]
                    new_data.append(this)
        
        self.df = pd.DataFrame(new_data, columns=new_cols).fillna(self.nan_value)
        
    def convert_string_values(self):
        yes_no = {"No": 0,
                "Sì": 1,
                "Si": 1,
                "no": 0,
                "sì": 1,
                "si": 1,
                "F": 1,
                "M": 0,
                "mancante": -1,
                 -1: -1,
                 -1.0: -1}
        le = preprocessing.LabelEncoder()
        for col in self.df.columns.values:
            if self.df[col].dtypes == "object_":
                unique_list = self.df[col].unique()
                if all([x in yes_no.keys() for x in unique_list]):
                    self.df[col] = self.df[col].map(yes_no)
#                 elif "data" not in col and len(unique_list) < 8:
                    
              #  if len(unique_list) < 8:
              #      enc_value_list = le.fit_transform(unique_list)
              #      col_map_dict = dict(zip([x+2 for x in enc_value_list], unique_list))
              #      self.df[col] = self.df[col].map(col_map_dict)
              #      self.map_dict[col] = col_map_dict
        

In [ ]:
foglio_grande = Plic_importer("C:\\Users\\julix\\webvalley_git\\full_project\\data\\PLIC-Milano\\plic-milano-foglio-grande.xlsx")

In [ ]:
foglio_grande.df = foglio_grande.excel_to_pandas_df()

In [ ]:
foglio_grande.df.head()

In [ ]:
foglio_grande.drop_empty_cols()

from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/t214oaWEu9s?start=66" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [ ]:
foglio_grande.visit_cols_to_rows()

In [ ]:
foglio_grande.df.head()

In [ ]:
foglio_grande.df.info()

In [ ]:
foglio_grande.convert_string_values()

In [ ]:
foglio_grande.df.head()

In [ ]:
for col in foglio_grande.df.columns.values:
    l = foglio_grande.df[col].unique()
    if len(l) <5 and foglio_grande.df[col].dtype == "object_":
        print("Values of column '{}' are: {}. --{}".format(col, l, [type(x) for x in l]))

In [ ]:
l = set()
for col in foglio_grande.df.columns.values:
    if "data" in col:
        for val in foglio_grande.df[col].unique():
            if type(val) not in [pd._libs.tslibs.timestamps.Timestamp, np.datetime64] :
                l.add(val)

In [ ]:
l